In [8]:
import numpy as np
from qiskit import QuantumCircuit, Aer
from qiskit.visualization import plot_histogram

def encode_message(bits, bases):
    message = []
    n = len(bits)  # Get the number of bits from the length of the input list
    for i in range(n):
        qc = QuantumCircuit(1, 1)
        if bases[i] == 0:  # Prepare qubit in Z-basis
            if bits[i] == 1:  # Change from 0 to 1 for consistency
                qc.x(0)
        else:  # Prepare qubit in X-basis
            if bits[i] == 1:  # Change from 0 to 1 for consistency
                qc.x(0)
            qc.h(0)
        qc.barrier()
        message.append(qc)
    return message

def measure_message(message, bases):
    backend = Aer.get_backend('aer_simulator')
    measurements = []
    n = len(message)  # Get the number of qubits from the length of the input list
    for q in range(n):
        if bases[q] == 0:  # measuring in Z-basis
            message[q].measure(0, 0)
        else:  # measuring in X-basis
            message[q].h(0)
            message[q].measure(0, 0)
        aer_sim = Aer.get_backend('aer_simulator')
        result = aer_sim.run(message[q], shots=1, memory=True).result()
        measured_bit = int(result.get_memory()[0])
        measurements.append(measured_bit)
    return measurements

def remove_garbage(a_bases, b_bases, bits):
    good_bits = [bit for a, b, bit in zip(a_bases, b_bases, bits) if a == b]
    return good_bits

def sample_bits(bits, selection):
    sample = [bits[i % len(bits)] for i in selection]
    return sample


In [9]:
# Set seed for random number generation for reproducibility
np.random.seed(seed=0)

n = 100
alice_bits = np.random.randint(2, size=n)
alice_bases = np.random.randint(2, size=n)

message = encode_message(alice_bits, alice_bases)

bob_bases = np.random.randint(2, size=n)
bob_results, bob_circuits_after_measurement = measure_message(message, bob_bases)

alice_key = remove_garbage(alice_bases, bob_bases, alice_bits)
bob_key = remove_garbage(alice_bases, bob_bases, bob_results)

sample_size = 15
bit_selection = np.random.randint(n, size=sample_size)
bob_sample = sample_bits(bob_key, bit_selection)
alice_sample = sample_bits(alice_key, bit_selection)

samples_match = bob_sample == alice_sample

print("Alice's bits:", alice_bits)
print("Alice's bases:", alice_bases)
print("Bob's bases:", bob_bases)
print("Bob's results:", bob_results)
print("Alice's key:", alice_key)
print("Bob's key:", bob_key)
print("Bob's sample:", bob_sample)
print("Alice's sample:", alice_sample)
print("Do samples match?", samples_match)
print("Key length =", len(alice_key))


Alice's bits: [0 1 1 0 1 1 1 1 1 1 1 0 0 1 0 0 0 0 0 1 0 1 1 0 0 1 1 1 1 0 1 0 1 0 1 1 0
 1 1 0 0 1 0 1 1 1 1 1 0 1 0 1 1 1 1 0 1 0 0 1 1 0 1 0 1 0 0 0 0 0 1 1 0 0
 0 1 1 0 1 0 0 1 0 1 1 1 1 1 1 0 1 1 0 0 1 0 0 1 1 0]
Alice's bases: [1 0 0 1 0 0 0 1 1 0 1 0 0 0 0 0 1 0 1 0 1 1 1 1 1 0 1 1 1 1 0 1 1 0 0 1 0
 0 0 0 1 1 0 0 1 0 1 1 1 1 0 0 0 1 0 1 1 1 0 1 0 0 1 0 1 1 0 0 1 0 1 0 1 0
 1 0 1 0 0 0 1 0 1 0 1 0 0 0 0 0 1 0 0 1 0 0 0 1 0 0]
Bob's bases: [1 0 1 0 0 1 1 0 0 0 1 1 0 0 0 0 0 1 0 1 0 0 0 1 1 1 0 0 1 1 1 1 0 0 0 1 1
 0 1 0 0 1 0 1 1 1 1 0 0 0 1 1 1 0 1 1 1 1 0 0 1 1 0 0 0 1 1 0 1 1 1 1 1 0
 0 0 1 0 1 0 1 1 0 0 0 1 0 0 1 1 1 1 0 1 0 0 0 0 1 1]
Bob's results: [0, 1, 1, 0, 1, 1, 0, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 0, 1, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0]
Alice's key: [1, 1, 0, 1, 0, 0